# Introduction to data science
Author: Gérard Lichtert

## Introduction
This notebook is to clean data from a csv, it removes unnecesary columns, computes means and saves the processed data to a new csv file found in the output folder.

It will also make a new dataframe containing averages per day per participant and save it to a csv for the OBSE survey

## Usage
Following the instructions in the README.md file is crucial for installation, for execution it suffices to run all twice. The first time will create the necessary directories. You are expected to put the input CSV files in the resources/in directory. After you've moved the CSV files to the resources/in directory you can run it again to process the CSV files. After processing they will end up in the resources/out directory and depending on the data processed in either obse or afvar subdirectories.

## Variables you can change
In the following code cells you can change the variables as you need as these will be the columns that need to be removed from the OBSE survey and the other one respectively.

In [95]:
# This is a list of headers we want to delete (exluding the ones with _TZ, _RT and _TZ) from the OBSE survey
# the headers with _TZ, _RT and _TZ will be removed automatically.
HEADERS_TO_DROP_FINAL_SURVEY: list[str] = [
    "Start Date",
    "End Date",
    "Response Type",
    "IP Address",
    "Progress",
    "Duration (in seconds)",
    "Finished",
    "Recorded Date",
    "Response ID",
    "Recipient Last Name",
    "Recipient First Name",
    "External Data Reference",
    "Location Latitude",
    "Location Longitude",
    "Distribution Channel",
    "User Language",
    'Recipient Email',
]

In [96]:
# This is a list of headers we want to delete (exluding the ones with _TZ, _RT and _TZ) from the afvar survey
# the headers with _TZ, _RT and _TZ will be removed automatically.
HEADERS_TO_DROP_OTHER = [
    "STUDY_ID",
    "STUDY_NAME",
    "STUDY_VERSION",
    "SURVEY_ID",
    "TRIGGER",
    "EXPORT_TZ",
    "START_END",
    "CREATED_TS",
    "SCHEDULED_TS",
    "STARTED_TS",
    "EXPIRED_TS",
    "TOTAL_RT",
    "RAND_PROB",
    "PARTICIPANT_TZ",
]

In [97]:
HEADERS_TO_DROP_FROM_DEMOGRAPHICS: list[str] = [
    "Start Date",
    "End Date",
    "Response Type",
    "IP Address",
    "Progress",
    "Duration (in seconds)",
    "Finished",
    "Recorded Date",
    "Response ID",
    "Recipient Last Name",
    "Recipient First Name",
    "Recipient Email",
    "External Data Reference",
    "Location Latitude",
    "Location Longitude",
    "Distribution Channel",
    "User Language",
    "Beste participant,\r\n\r\nHartelijk dank voor uw deelname aan dit onderzoek.\r\n\r\n\r\n\r\nHet onderzoek\r\n\r\nhet onderzoek bestaat uit twee delen en peilt naar de relatie met uw werk. Deze enquête vormt het eerste deel van het onderzoek. De enquête bevat tien vragen en neemt ongeveer één minuut in beslag. Deze enquête gaat na of u in aanmerking komt voor het tweede deel van het onderzoek betreffende de relatie met uw werk. We vragen uw e-mailadres om u vervolgens een uitnodiging tot de app SEMA3 te sturen. Via deze app zal u het tweede deel van het onderzoek kunnen vervolledigen. \r\n\r\n\r\n\r\nProcedure\r\n\r\nHet onderzoek zelf of het tweede deel betreft een dagboekonderzoek. Hierbij zal u 20 werkdagen lang om 18h een vragenlijst krijgen, waarin u bevraagd zal worden over uw werkdag. Het invullen van deze vragenlijst zal niet langer duren dan 1 minuut. U heeft de tijd tot 24h om deze vragenlijst in te vullen. Participanten die aan 80% of meer meetmomenten deelnamen krijgen een vergoeding voor hun participatie.\r\n\r\n\r\n\r\nWat gebeurt er met mijn gegevens?\r\n\r\nDe verzamelde gegevens worden alleen gebruikt voor wetenschappelijke doeleinden en worden vertrouwelijk behandeld in overeenstemming met de Europese Algemene Verordening Gegevensbescherming (GDPR). Uw deelname is vrijwillig en u kunt op elk moment beslissen om het onderzoek zonder het geven van een reden te beëindigen. Voor meer informatie over uw rechten en de behandeling van de gegevens kunt u contact opnemen met de verantwoordelijke afdeling aan de VUB (dpo@vub.be).\r\n\r\n\r\n\r\nContact\r\n\r\nAls u vragen en/of opmerkingen hebt over dit onderzoek, kunt u ze hieronder meegeven of contact opnemen met Sam de Pape (Sam.De.Pape@vub.be) of Jules Joukes (Jules.Sabine.P.Joukes@vub.be), of met superviserend professor Joeri Hofmans (Joeri.Hofmans@vub.be).",
    "Door deze enquête in te vullen, ga ik akkoord met mijn deelname aan dit onderzoek en met de verwerking van mijn persoonlijke gegevens in overeenstemming met de Europese Algemene Verordening Gegevensbescherming (GDPR) door de onderzoekers van de VUB.",
    "Gaat u volgende maand minstens één week op vakantie? - Selected Choice",
    "Gaat u volgende maand minstens één week op vakantie? - Andere, namelijk: - Text",
]

In [98]:
OBSE_COLUMNS = [
    "PARTICIPANT_ID",
    "UPLOADED_TS",
    "ACTIVITEIT",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    "10",
]


## Some libraries and definitions (do not change)


In [99]:
from pathlib import Path
import polars as pl

resources = Path("../resources/in/")
out = Path("../resources/out/")


def write_csv_and_excel(lf, filename):
    lf.sink_csv(f"{out.as_posix()}/{filename}.csv")
    lf.collect().write_excel(f"{out.as_posix()}/{filename}.xlsx")


## reading and cleaning the data

In [100]:
final_survey_lf: pl.LazyFrame = pl.scan_csv(f"{resources.as_posix()}/final_survey.csv")
sema_lf: pl.LazyFrame = pl.scan_csv(
    resources.as_posix() + "/" + "data_uit_SEMA3_OBSE_en_laatste_survey.csv"
)
demographics_lf: pl.LazyFrame = pl.scan_csv(
    resources.as_posix() + "/" + "demografische_gegevens_eerste_survey.csv"
)
keys: pl.LazyFrame = pl.scan_csv(
    f"{resources.as_posix()}/identificatie_key.csv", separator=";"
)


def remove_headers(
    lf: pl.LazyFrame, headers: list[str], del_timed_headers=True
) -> pl.LazyFrame:
    columns: list[str] = lf.columns
    keep = [col for col in columns if col not in headers]
    if del_timed_headers:
        keep = [
            col
            for col in keep
            if not col.endswith("_TZ")
            and not col.endswith("_RT")
            and not col.endswith("_TZ")
        ]
    return lf.select(keep)


demographics_lf = remove_headers(
    demographics_lf, HEADERS_TO_DROP_FROM_DEMOGRAPHICS, False
)
final_survey_lf = remove_headers(final_survey_lf, HEADERS_TO_DROP_FINAL_SURVEY, False)
sema_en_obse_lf = remove_headers(sema_lf, HEADERS_TO_DROP_OTHER)
obse_lf: pl.LazyFrame = sema_en_obse_lf.filter(pl.col("SURVEY_NAME") == "OBSE")
werktevredenheid_lf: pl.LazyFrame = sema_en_obse_lf.filter(
    pl.col("SURVEY_NAME") == "werktevredenheid/ SWLS/ PRESTATIE"
).filter(pl.col("UPLOADED_TS").is_not_null())

In [101]:
obse_lf: pl.LazyFrame = (
    obse_lf.select(OBSE_COLUMNS)
    .filter(pl.col("ACTIVITEIT") == "1")
    .cast({str(integer): pl.UInt8 for integer in range(1, 11)})
)
valid_obse_participants: list[str] = (
    obse_lf.group_by("PARTICIPANT_ID")
    .len()
    .filter(pl.col("len") >= 5)
    .select("PARTICIPANT_ID")
    .unique()
    .collect()
    .to_dict(as_series=False)["PARTICIPANT_ID"]
)
obse_lf = (
    obse_lf.filter(pl.col("PARTICIPANT_ID").is_in(valid_obse_participants))
    .with_columns(
        (
            (
                pl.col("1")
                + pl.col("2")
                + pl.col("3")
                + pl.col("4")
                + pl.col("5")
                + pl.col("6")
                + pl.col("7")
                + pl.col("8")
                + pl.col("9")
                + pl.col("10")
            )
            / 10
        ).alias("MEAN")
    )
    .sort("UPLOADED_TS")
)
write_csv_and_excel(obse_lf, "obse_with_daily_mean")
obse_participant_mean = obse_lf.group_by("PARTICIPANT_ID").agg(pl.col("MEAN").mean())
write_csv_and_excel(obse_participant_mean, "obse_participant_mean")

In [102]:
final_survey: pl.LazyFrame = (
    final_survey_lf.join(
        keys, left_on="Wat is uw e-mailadres dat u opgaf voor SEMA3?", right_on="Email"
    )
    .rename(
        {
            "Wat is uw e-mailadres dat u opgaf voor SEMA3?": "EMAIL",
            "Id": "PARTICIPANT_ID",
        }
    )
    .drop(["EMAIL"])
)


def first_variable(header):
    lst: list[str] = [f"_{i}" for i in range(1, 6)]
    res = False
    for el in lst:
        if header.endswith(el):
            res = True
            break
    return res


FINAL_SURVEY_IDENTIFIERS = ["PARTICIPANT_ID", "UPLOADED_TS", "COMPLETED_TS"]
FINAL_SURVEY_WERKTEVREDENHEID_QUESTIONS = []
for label in werktevredenheid_lf.columns:
    if first_variable(label):
        FINAL_SURVEY_WERKTEVREDENHEID_QUESTIONS.append(label)
FINAL_SURVEY_SATISFACTION_WITH_LIFE_QUESTIONS: list[str] = [
    "IDEAAL",
    "OMSTANDIGHEDEN",
    "TEVREDEN",
    "BELANGRIJKE_DINGEN",
    "NIETS_VERANDEREN",
]
FINAL_SURVEY_PRESTATIE_QUESTIONS: list[str] = []
werktevredenheid_eerste_variabele_lf: pl.LazyFrame = (
    werktevredenheid_lf.select(
        FINAL_SURVEY_IDENTIFIERS + FINAL_SURVEY_WERKTEVREDENHEID_QUESTIONS
    )
    .cast(
        {
            FINAL_SURVEY_WERKTEVREDENHEID_QUESTIONS[i]: pl.UInt8
            for i in range(len(FINAL_SURVEY_WERKTEVREDENHEID_QUESTIONS))
        }
    )
    .filter(pl.col("COMPLETED_TS").is_not_null() & pl.col("UPLOADED_TS").is_not_null())
)
werktevredenheid_tweede_variabele_lf: pl.LazyFrame = (
    werktevredenheid_lf.select(
        FINAL_SURVEY_IDENTIFIERS + FINAL_SURVEY_SATISFACTION_WITH_LIFE_QUESTIONS
    )
    .filter(pl.col("COMPLETED_TS").is_not_null() & pl.col("UPLOADED_TS").is_not_null())
    .cast(
        {
            FINAL_SURVEY_SATISFACTION_WITH_LIFE_QUESTIONS[i]: pl.UInt8
            for i in range(len(FINAL_SURVEY_SATISFACTION_WITH_LIFE_QUESTIONS))
        }
    )
)
DERDE_VARIABELEN_TO_DROP_COLUMNS = (
    FINAL_SURVEY_SATISFACTION_WITH_LIFE_QUESTIONS
    + FINAL_SURVEY_WERKTEVREDENHEID_QUESTIONS
    + [str(i) for i in range(1, 11)]
)
for label in werktevredenheid_lf.columns:
    if (
        label.endswith("_0")
        or label == "SURVEY_NAME"
        or "INLEIDING" in label
        or "CLONE" in label
        or ")" in label
        or label == "D"
        or label == "ACTIVITEIT"
        or label == "BEDANKT"
        or label == "SLOT"
        or label == "INTRO"
        or label == "CONTROLLEVRAAG"
        or label == "TAAKPRESTATIE"
    ):
        DERDE_VARIABELEN_TO_DROP_COLUMNS.append(label)
werktevredenheid_derde_variabele_lf = werktevredenheid_lf.drop(
    DERDE_VARIABELEN_TO_DROP_COLUMNS
).filter(pl.col("COMPLETED_TS").is_not_null() & pl.col("UPLOADED_TS").is_not_null())
werktevredenheid_headers = werktevredenheid_derde_variabele_lf.columns[3:]
werktevredenheid_derde_variabele_lf = werktevredenheid_derde_variabele_lf.cast(
    {
        werktevredenheid_headers[i]: pl.UInt8
        for i in range(len(werktevredenheid_headers))
    }
)

In [103]:
# Compute the weighted sum
def compute_weighted_sum(lf):
    cols = []
    for i in FINAL_SURVEY_WERKTEVREDENHEID_QUESTIONS:
        string = i.split("_")[0]
        if string not in cols:
            cols.append(string)
    for i in cols:
        lf = lf.with_columns(
            (
                (
                    pl.col(f"{i}_1")
                    + pl.col(f"{i}_2") * 2
                    + pl.col(f"{i}_3") * 3
                    + pl.col(f"{i}_4") * 4
                    + pl.col(f"{i}_5") * 5
                )
            ).alias(i)
        )
    return lf

werktevredenheid_eerste_variabele_lf = compute_weighted_sum(werktevredenheid_eerste_variabele_lf).drop(FINAL_SURVEY_WERKTEVREDENHEID_QUESTIONS)

In [105]:
def compute_mean(lf, from_index):
    mean_expr = pl.col(lf.columns[from_index])
    for column in lf.columns[from_index + 1 :]:
        mean_expr += pl.col(column)
    mean_expr /= len(lf.columns[from_index:])
    return lf.with_columns(mean_expr.alias("MEAN"))

werktevredenheid_eerste_variabele_lf = compute_mean(
    werktevredenheid_eerste_variabele_lf, 3
)
werktevredenheid_tweede_variabele_lf = compute_mean(
    werktevredenheid_tweede_variabele_lf, 3
)

werktevredenheid_derde_variabele_lf = compute_mean(
    werktevredenheid_derde_variabele_lf, 3
)
#werktevredenheid_eerste_variabele_lf.collect().head()

PARTICIPANT_ID,UPLOADED_TS,COMPLETED_TS,WERKOMSTANDIGHEDEN,WERKMETHODE,COLLEGA'S,ERKENNING,BAAS,VERANTWOORDELIJKHEID,SALARIS,CAPACITEITEN,ARBEIDSVERHOUDING,PROMOTIE,LEIDING,SUGGESTIES,WERKUREN,VARIATIE,WERKZEKERHEID,TOTAAL,MEAN
str,str,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,f64
"""s257136850""","""08-May-2024 10:03""","""08-May-2024 10:03""",2,1,2,5,1,2,3,2,2,4,2,2,1,2,1,2,2.125
"""s272728106""","""08-May-2024 17:04""","""08-May-2024 17:04""",3,3,2,2,3,1,3,2,4,4,2,3,3,2,1,2,2.5
"""s362909936""","""08-May-2024 19:03""","""08-May-2024 19:03""",1,2,1,1,1,2,2,1,1,2,1,2,2,2,1,1,1.4375
"""s665803620""","""08-May-2024 12:50""","""08-May-2024 12:50""",2,1,1,1,1,1,2,1,2,3,2,1,2,1,2,1,1.5
"""s935003383""","""08-May-2024 13:53""","""08-May-2024 13:53""",2,1,2,3,2,1,3,2,2,1,3,2,4,2,2,2,2.125
